
# Micromagnetic standard problem 1

**Author**: Marijan Beg

**Date**: 10 May 2016

## Problem specification

The simulated sample is a thin film cuboid with dimensions:
- length $L_{x} =  2 \,\mu\text{m}$,
- width $L_{y} = 1 \,\mu\text{m}$, and
- thickness $t = 20 \,\text{nm}$.

The material parameters (similar to permalloy) are:

- exchange energy constant $A = 1.3 \times 10^{-11} \,\text{J/m}$,
- magnetisation saturation $M_\text{s} = 8 \times 10^{5} \,\text{A/m}$.

Apart from the symmetric exchange and demagnetisation energies, uniaxial anisotropy energy is also present with

- $K = 0.5 \times 10^{3} \,\text{J/m}^{3}$
- $(1, 0, 0)$ direction.

More details on standard problem 1 specifications can be found in Ref. 1.

## Simulation

In the first step, we define all simulation parameters.

In [1]:
!rm -rf standard_problem1

In [2]:
import numpy as np

Lx = 2e-6  # x dimension of the sample(m)
Ly = 1e-6  # y dimension of the sample (m)
thickness = 20e-9  # sample thickness (m)
dx = dy = dz = 20e-9 # discretisation in x and y directions (m)

# Minimum sample coordinate.
p1 = (0, 0, 0)
# Maximum sample coordinate.
p2 = (Lx, Ly, thickness)
# Discretisation.
cell = (100e-9, 100e-9, dz)

Ms = 8e5  # saturation magnetisation (A/m)
A = 1.3e-11  # exchange energy constant (J/m)
K = 0.5e3  # uniaxial anisotropy constant (J/m**3)
u = (1, 0, 0)  # uniaxial anisotropy axis

After all required simulation parameters are defined, necessary modules are imported.

In [3]:
import discretisedfield as df
import oommfc as oc

Now, we can create atlas, mesh, and simulation objects.

In [4]:
# Create a mesh object.
mesh = oc.Mesh(p1, p2, cell)

# Create a system object.
system = oc.System(name='standard_problem1')

In the next step, energy terms are added. We add symmetric exchange, demagnetisation, and Zeeman energies to the system's Hamiltonian. At this point, we add a Zeeman energy contribution with zero external magnetic field, but this value will be successivelly changed in the hysteresis loop simulation.

In [5]:
system.mesh = mesh

system.hamiltonian = oc.Exchange(A) + oc.UniaxialAnisotropy(K, u) + oc.Demag()

The system is initialised in the $(10, 1, 0)$ direction [1].

In [6]:
system.m = df.Field(mesh, value=(10, 1, 0), normalisedto=Ms)

Before the hysteresis simulations are carried out, external magnetic field values at which the system should be relaxed must be created.

In [14]:
Hmax = (50e-3/oc.mu0, 0.87275325e-3/oc.mu0, 0)
Hmin = (-50e-3/oc.mu0, -0.87275325e-3/oc.mu0, 0)
n = 10

Finally, the system is relaxed at every external magnetic field value and average magnetisation is computed.

In [15]:
hd = oc.HysteresisDriver()
hd.drive(system, Hmax=Hmax, Hmin=Hmin, n=n)

## Hysteresis loop plot

After obtaining the average magnetisation at different external magnetic field values, hysteresis loop is plotted.

In [16]:
system.dt

,CGEvolve::MaxmxHxm,CGEvolve::Totalenergy,CGEvolve::DeltaE,CGEvolve::Bracketcount,CGEvolve::Linemincount,CGEvolve::Conjugatecyclecount,CGEvolve::Cyclecount,CGEvolve::Cyclesubcount,CGEvolve::Energycalccount,Eex,...,UZeeman::B,UZeeman::Bx,UZeeman::By,UZeeman::Bz,iteration,stage_iteration,stage,mx,my,mz
0,0.009087,-1.362111e-15,-3.944305e-30,42.0,28.0,1.0,33.0,33.0,71.0,1.583804e-18,...,50.007616,-50.0,-0.872753,0.0,54.0,54.0,0.0,-0.985542,0.028812,0.0
1,0.009441,-1.047951e-15,-2.563798e-30,101.0,69.0,2.0,78.0,44.0,172.0,2.263444e-18,...,40.006093,-40.0,-0.698203,0.0,133.0,78.0,1.0,-0.979213,0.039984,0.0
2,0.008426,-7.359258e-16,-4.437343e-30,154.0,102.0,3.0,117.0,38.0,259.0,2.779953e-18,...,30.004570,-30.0,-0.523652,0.0,201.0,67.0,2.0,-0.972199,0.050986,0.0
3,0.005833,-4.270953e-16,-2.021456e-30,255.0,149.0,5.0,191.0,23.0,408.0,2.782873e-18,...,20.003047,-20.0,-0.349101,0.0,319.0,117.0,3.0,-0.958661,0.076314,0.0
4,0.007904,-1.242627e-16,-2.662406e-30,337.0,195.0,7.0,258.0,16.0,537.0,5.369664e-18,...,10.001523,-10.0,-0.174551,0.0,431.0,111.0,4.0,-0.932398,0.079301,0.0
5,0.009846,1.637730e-16,-6.409495e-31,446.0,246.0,10.0,339.0,27.0,698.0,1.292115e-17,...,0.000000,0.0,0.000000,0.0,565.0,133.0,5.0,-0.856609,0.041178,0.0
6,0.009062,-2.189946e-17,-2.215590e-30,1303.0,473.0,107.0,744.0,49.0,1783.0,4.055846e-17,...,10.001523,10.0,0.174551,0.0,1496.0,930.0,6.0,0.573883,0.013760,0.0
7,0.009489,-2.069132e-16,-2.144716e-30,1364.0,512.0,108.0,790.0,45.0,1884.0,4.050007e-17,...,20.003047,20.0,0.349101,0.0,1577.0,80.0,7.0,0.581551,0.015239,0.0
8,0.006395,-3.940737e-16,-3.451266e-30,1411.0,544.0,109.0,826.0,35.0,1964.0,4.041539e-17,...,30.004570,30.0,0.523652,0.0,1638.0,60.0,8.0,0.587446,0.016678,0.0
9,0.008808,-5.829903e-16,-2.563798e-30,1450.0,568.0,110.0,856.0,29.0,2028.0,4.021973e-17,...,40.006093,40.0,0.698203,0.0,1686.0,47.0,9.0,0.592629,0.018404,0.0


In [9]:
import matplotlib.pyplot as plt

%matplotlib inline

plt.figure(figsize=(10,8))
plt.plot(H_array*oc.mu0, mx_average, label='<mx>')
plt.plot(H_array*oc.mu0, my_average, label='<my>')
plt.plot(H_array*oc.mu0, mz_average, label='<mz>')
plt.xlim(-Hmax*oc.mu0, Hmax*oc.mu0)
plt.xlabel(r'$\mu_{0}$H (T)')
plt.ylabel('<m>')
plt.grid()

NameError: name 'H_array' is not defined

## References

[1] http://www.ctcms.nist.gov/~rdm/mumag.org.html